##### Cleaning Train and Test Dataset

In this notebook, we will clean the train and test dataset

In [1]:
#Import Libraries

import numpy as np
import pandas as pd
import time
import datetime as datetime

### Import and Inspect Data

The train dataset has 10,506 entries and 12 columns while the test dataset has 116,293 entries and 11 columns. The NumMosquitos and WnvPresent are missing from the test dataset and these are our predictor values (i.e. y). The test dataset also has a ID column, to identify every entry.
Several of these columns such as Address, Block and AddressNumberAndStreet pertain to location details. We decided to retain only the latitude and longitude columns and drop all other location related features for both train and test datasets as they are deemed redundant.

In [2]:
train = pd.read_csv('./datasets/train.csv')
test = pd.read_csv('./datasets/test.csv')

In [3]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [4]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


### Clean Data

We created functions to rename the columns and drop the redundant location related features. We also decided to split the date column into year, month and day columns so that it will be easier for us to analyse the seasonality effect later on.
As explained in the Kaggle competition, the train and test datasets are organised in such a way that when the number of mosquitos exceed 50, they are split into another record (another row in the dataset), such that the number of mosquitos are capped at 50. We have thus also ran a for-loop to combine these duplicate rows and sum up the NumMosquitos.
We then combined both train and test datasets and saved them as csv files for exploratory data analysis later on.

In [5]:
def create_yr(x): 
    return x.split('-')[0] 

def create_mth(x): 
    return x.split('-')[1] 

def create_day(x): 
    return x.split('-')[2] 

def rename_columns (columns):
    return [column.lower() for column in columns]

def clean_data(df): 
    df['year'] = df.Date.apply(create_yr)
    df['month'] = df.Date.apply(create_mth)
    df['day'] = df.Date.apply(create_day)    

    df.drop(['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'Date'], axis = 1, inplace = True)
    df.columns = rename_columns(df.columns)
    
    return df

In [6]:
train = clean_data(train)
test = clean_data(test)

In [7]:
train.head()

,species,block,street,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29
2,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,1,0,2007,05,29
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,1,0,2007,05,29
4,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,4,0,2007,05,29


In [35]:
train_2.iterrows()

<generator object DataFrame.iterrows at 0x7fef80a91120>

In [12]:
# merge nummosquitos and wnvpresent for duplicated rows

# create train_2 (new copy)
# to compare train and train_2 and verify code
train_2 = train.copy()

# duplicated rows have the same values for the columns below
cols = ['species','trap','year','month', 'day','latitude','longitude']

for row_idx,row in train_2.iterrows():
    
    # skip 1st row
    if row_idx > 0:
        
        # reset counter for each row
        duplicate_count = 0        
        
        for col in cols:
            
            # compare cells in current and previous rows
            # increment counter if both cells have the same value 
            if train_2.at[row_idx,col] == train_2.at[row_idx-1,col]:
                duplicate_count += 1               
                
        # counter equal to number of selected column
        # current row['nummosquitos','wnvpresent'] = sum of current and previous rows
        if duplicate_count == len(cols):
            train_2.at[row_idx,'nummosquitos'] = train_2.at[row_idx,'nummosquitos'] + train_2.at[row_idx-1,'nummosquitos']
            train_2.at[row_idx,'wnvpresent'] = train_2.at[row_idx,'wnvpresent'] + train_2.at[row_idx-1,'wnvpresent']

# remove duplicated rows (keep only last row)
train_2.drop_duplicates(subset=cols, keep='last',inplace=True)

In [36]:
# check total nummosquitos and wnvpresent

print(train['nummosquitos'].sum())
print(train['wnvpresent'].sum())
print('')

print(train_2['nummosquitos'].sum())
print(train_2['wnvpresent'].sum())
print('')

print(train_2['wnvpresent'].unique())

135039
551

135039
457

[0 1]


In [14]:
# convert wnvpresent: total count to 0 or 1

for row_idx,row in train_2.iterrows():
    
    if row['wnvpresent'] > 1:
        #print(row_idx,row['wnvpresent'])
        train_2.at[row_idx,'wnvpresent'] = 1
        
print(train_2['wnvpresent'].sum())
print(train_2['wnvpresent'].unique())

457
[0 1]


#### Combine df_train and df_test

In [37]:
# show dataframe info

print(train_2.shape)
print(test.shape)
print('')

print(train_2.columns)
print('')

print(test.columns)

(8475, 11)
(116293, 10)

Index(['species', 'block', 'street', 'trap', 'latitude', 'longitude',
       'nummosquitos', 'wnvpresent', 'year', 'month', 'day'],
      dtype='object')

Index(['id', 'species', 'block', 'street', 'trap', 'latitude', 'longitude',
       'year', 'month', 'day'],
      dtype='object')


In [16]:
# make deep copy
train_3 = train_2.copy()
test_3 = test.copy() # copy to test_3 instead of test_2

# train: missing id column
# add id column
train_3['is_train'] = 1
train_3['id'] = -1

# test: missing nummosquitos and wnvpresent columns
# add this columns
test_3['is_train'] = 0
test_3['nummosquitos'] = -1
test_3['wnvpresent'] = -1

In [17]:
# check train

train_3.head()

,species,block,street,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day,is_train,id
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29,1,-1
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29,1,-1
2,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,1,0,2007,05,29,1,-1
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,1,0,2007,05,29,1,-1
4,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,4,0,2007,05,29,1,-1


In [19]:
# check test

test_3.head()

,id,species,block,street,trap,latitude,longitude,year,month,day,is_train,nummosquitos,wnvpresent
0,1,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11,0,-1,-1
1,2,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11,0,-1,-1
2,3,CULEX PIPIENS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11,0,-1,-1
3,4,CULEX SALINARIUS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11,0,-1,-1
4,5,CULEX TERRITANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11,0,-1,-1


In [24]:
# combine train and test

train_test = pd.concat([train_3,test_3])

In [25]:
# check train_test

train_test.head()

,species,block,street,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day,is_train,id
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29,1,-1
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29,1,-1
2,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,1,0,2007,05,29,1,-1
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,1,0,2007,05,29,1,-1
4,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,4,0,2007,05,29,1,-1


#### Output Data

In [26]:
train_2.head()

,species,block,street,trap,latitude,longitude,nummosquitos,wnvpresent,year,month,day
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.954690,-87.800991,1,0,2007,05,29
2,CULEX RESTUANS,62,N MANDELL AVE,T007,41.994991,-87.769279,1,0,2007,05,29
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,1,0,2007,05,29
4,CULEX RESTUANS,79,W FOSTER AVE,T015,41.974089,-87.824812,4,0,2007,05,29


In [27]:
test.head()

,id,species,block,street,trap,latitude,longitude,year,month,day
0,1,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11
1,2,CULEX RESTUANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11
2,3,CULEX PIPIENS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11
3,4,CULEX SALINARIUS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11
4,5,CULEX TERRITANS,41,N OAK PARK AVE,T002,41.95469,-87.800991,2008,06,11


In [28]:
train_2.to_csv('./clean data/train_clean.csv',index=False)
test.to_csv('./clean data/test_clean.csv',index=False)
train_test.to_csv('./clean data/train_test_clean.csv',index=False)

In [38]:
train_2.shape

(8475, 11)

In [39]:
test.shape

(116293, 10)

In [40]:
train_test.shape

(124768, 13)